## Data preprocess

In [356]:
from pathlib import Path
import pandas as pd



def read_data(filename):
    df = pd.read_excel(filename)
    return df.to_numpy()

In [357]:
import numpy as np

## UNUSE
def one_hot_encoding(arr):
    s = set()
    _sd = dict()
    for e in arr:
        s.add(e)

    sl = list(s)
    for i in range(len(sl)):
        encode = np.zeros(len(sl))
        encode[i] = 1
        _sd[sl[i]] = encode
    
    return _sd


def data_preprocess(filepath):
    data = read_data(filepath)

    # switch how many workers
    dataset = dict()

    for d in data:
        ip_address = d[-4]
        worker_data_set = np.concatenate((d[:-4], d[-4 + 1:])).tolist()
        if ip_address not in dataset:
            dataset[ip_address] = list()
            dataset[ip_address].append(worker_data_set)
        else:
            dataset[ip_address].append(worker_data_set)

    # ip_data = one_hot_encoding(list(dataset.keys()))

    # for key in dataset.keys():
    #     for i in range(len(dataset[key])):
    #         dataset[key][i] += ip_data[key].tolist()

    return dataset

In [358]:
filename = "nodeWaitTime_jobsNumber_v12_500000_test"

resutls_dir = Path.cwd().parent / "training_data"

# Data preprocessing
file_path = f"{resutls_dir}\\{filename}.xlsx"

dataset = data_preprocess(file_path)

## DATA Style View

In [359]:
# dataset is a dictionary

for key, data in dataset.items():
    print(f"{key}: {data}")

    # data array style view
    print(np.array(data).shape)

192.168.0.151: [[8.643675804138184, 500000, 8.260582685470581, 0.02809000015258789, 77], [8.61923098564148, 500000, 8.243952512741089, 0.02257037162780762, 77], [16.89937806129456, 500000, 8.25963544845581, 8.27843189239502, 73], [49.96948528289795, 500000, 8.246594667434692, 41.36457467079163, 72], [33.48862743377686, 500000, 8.25572919845581, 24.87043642997742, 74], [16.91488528251648, 500000, 8.285682916641235, 8.272102117538452, 76], [49.92246627807617, 500000, 8.229937076568604, 41.34016847610474, 69], [41.70260190963745, 500000, 8.307250499725342, 33.04615187644958, 73], [82.97169256210327, 500000, 8.265699625015259, 74.34708857536316, 65], [66.48955368995667, 500000, 8.249054193496704, 57.87806272506714, 70], [66.48756003379822, 500000, 8.288204431533813, 57.84411334991455, 70], [66.43909692764282, 500000, 8.243295907974243, 57.83712983131409, 67], [91.23776698112488, 500000, 8.262758016586304, 82.61545038223267, 67], [82.98885536193848, 500000, 8.236354351043701, 74.39715671539

## XGBOOST - 1

In [360]:
from xgboost import XGBRegressor

# Create XGBoost regression model
model = XGBRegressor(
    objective="reg:squarederror",   # Loss function to minimize: squared error for regression  损失函数为平方损失函数, 
    n_estimators=100,  # Number of boosting rounds (number of trees)  迭代次数，即基础学习器的数量
    learning_rate=0.1,  # Boosting learning rate (controls the step size)  学习率，控制每次更新的步长
    max_depth=5,  # Maximum depth of a tree  树的最大深度
    min_child_weight=1,  # Minimum sum of instance weight (hessian) needed in a child  叶子节点最小权重
    subsample=0.8,  # Subsample ratio of the training instance (randomly sampled)  每棵树随机采样的比例
    colsample_bytree=0.8,  # Subsample ratio of columns when constructing each tree  每棵树随机选择的特征比例
    alpha=0.01,  # L1 regularization term on weights  L1 正则化项参数
    reg_lambda=0.01,  # L2 regularization term on weights  L2 正则化项参数
)

In [361]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge

# X jobs number
# y wait time


# select key
ip_key_index = 2
ip_key = list(dataset.keys())[ip_key_index]


# exclude outliers
for key, data in dataset.items():
    for d in data:
        if (d[2] + d[3]) / d[0] < 0.3:
            data.remove(d)


# select value from dataset
X, y = np.array(dataset[ip_key])[:, -1:], np.array(dataset[ip_key])[:, -2:-1]

train_size = int(len(X) * 0.7)

# 拆分数据集为训练集和测试集
# split dataset to train dataset and test dataset

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=35
)

# time series
# X_train = X[:train_size]
# X_test = X[train_size:]
# y_train = y[:train_size]
# y_test = y[train_size:]

# model fit
model.fit(X_train, y_train)

XGBRegressor(alpha=0.01, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [362]:
from sklearn.metrics import mean_absolute_error

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mea = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mea}")


print(f"    real    |    pred   \n")
for i in range(len(y_test)):
    print(f"    {y_test[i]}     |     {y_pred[i]}      ")

Mean Squared Error: 5110.638411456521
Mean Absolute Error: 55.26937943696975
    real    |    pred   

    [16.54714155]     |     33.08781814575195      
    [90.84541535]     |     1.8472654819488525      
    [132.15499473]     |     -0.28707045316696167      
    [33.12422323]     |     33.08781814575195      
    [165.15574336]     |     101.44715881347656      
    [41.35717583]     |     33.08781814575195      
    [165.15912437]     |     88.26870727539062      


## XGBRegressor

In [363]:
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error


# 创建 XGBoost 回归模型
model = XGBRegressor(
    objective="reg:squarederror",
    n_estimators=100,
    learning_rate=0.1,
    tree_method="hist",
    device="cuda",
)

# Define the parameter grid for GridSearchCV
param_grid = {
    "max_depth": [3],  # Range of maximum depth of trees to explore  扩展最大深度的范围
    "learning_rate": [0.1],  # Range of learning rates to explore  扩展学习率的范围
    "n_estimators": [100],  # Range of number of trees (boosting rounds) to explore  扩展树的数量范围
    "gamma": [0],   # Range of gamma parameter to explore (controls tree split) 扩展 gamma 参数范围
    "subsample": [1],  # Range of subsample ratios to explore  扩展子样本比例范围
    "colsample_bytree": [0.8, 0.7],  # Range of column subsample ratios for each tree to explore  扩展列采样比例范围
    "reg_alpha": [0, 0.5],  # Range of L1 regularization parameters to explore  扩展 L1 正则化参数范围
    "reg_lambda": [0, 0.5],  # Range of L2 regularization parameters to explore  扩展 L2 正则化参数范围
}


grid_search = GridSearchCV(
    estimator=model, param_grid=param_grid, cv=5, scoring="neg_mean_squared_error"
)
grid_search.fit(X_train, y_train)

print("Best parameters found: ", grid_search.best_params_)

# 使用最佳参数重新训练模型
# Re-train the model using the best parameters
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train, verbose=True)


Best parameters found:  {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.5, 'reg_lambda': 0.5, 'subsample': 1}


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device='cuda', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [364]:
from openpyxl import Workbook
from pathlib import Path

def to_excel(data, filename, sheet_style):
    workbook = Workbook()
    sheet = workbook.active

    # | user-response-time | request-number | response-ip   | process-time  |
    # | 0.3                | 10000          | 192.168.0.150 | 14.523432     |
    # | 0.5                | 20000          | 192.168.0.151 | 9.5232642     |

    sheet.append(sheet_style)

    for row in data:
        sheet.append(row)

    results_dir = str(Path.cwd().parent / "results")

    workbook.save(filename=f"{results_dir}\\{filename}.xlsx")

In [365]:
preds = best_model.predict(X_test)

rmse = mean_squared_error(y_test, preds, squared=False)
print(f"RMSE: {rmse}")

mae = mean_absolute_error(y_test, preds)
print(f"MAE: {mae}")


avg = np.mean(y_test)

print(f"AVG: {avg}")

# write into excal

sheet_style = ["test", "prediction", "difference", "accuracy"]

data_list = list()


for i in range(len(y_test)):
    acc = 0
    diff = round(abs(y_test[i][0] - preds[i]), 6)
    if preds[i] < 0:
        acc = 0
    else:
        rate = round(diff / y_test[i][0], 4)
        if rate <= 1 or rate >= 0:
            acc = 1 - rate
        if rate > 1:
            acc = 0

    data_list.append(
        [
            y_test[i][0],
            preds[i],
            diff,
            f"{round(acc * 100, 2)}%",
        ],
    )


filename = "nodeWaitTime_requestJobs_Prediction_v13_test_ip2"
to_excel(data_list, filename, sheet_style)


print("Results saved finished.")

RMSE: 69.19960729054657
MAE: 53.33056228501455
AVG: 92.04911691801888
Results saved finished.


d:\model_fit\.venv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
